This script succeeds DS_1_Generate_Dataset_for_Analysis.py and performs the GridSearch using cross validation on AdaBoost model to determine the optimal parameters of AdaBoost associated with each dataset with accuracy as the metric

In [1]:
import os
import pandas as pd

# preprocessing
from sklearn.model_selection import train_test_split

# Modelling algorithm
from sklearn.ensemble import AdaBoostClassifier

# Model selection
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV, cross_val_score

#metrics
from sklearn.metrics import roc_curve,auc,accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import classification_report


In [2]:
def FindOptimalParm(X,y):    
    # Adaboost classifiers - find the optimal parameters using unscaled raw data 

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,stratify=y)

    gs_parm_grid = {'n_estimators':[100,200,500,800,1000,1500],
                   'learning_rate':[0.01,0.05,0.1,0.2,0.3]}
    ab_clf = AdaBoostClassifier()
    cv_model = RepeatedStratifiedKFold(n_splits = 5, n_repeats=1,random_state=1)
    grid_search_model_raw = GridSearchCV(estimator = ab_clf, 
                                     param_grid = gs_parm_grid, 
                                     n_jobs=1,
                                     cv=cv_model, 
                                     scoring = 'accuracy',
                                     verbose = 2)

    grid_search_score_raw = grid_search_model_raw.fit(X,y)

    # cv_scores = cross_val_score(ab_clf, X,y,scoring='accuracy',cv=cv_model,n_jobs=1,error_score='raise')

    # print('Accuracy: %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

    print(grid_search_score_raw)
    print("Best scores:")
    print("Best: %f using %s" % (grid_search_score_raw.best_score_, grid_search_score_raw.best_params_))
    return grid_search_score_raw.best_score_, grid_search_score_raw.best_params_

In [17]:
#root_path = "Data\Model Datasets" # uncomment this line later
root_path = "D:\PhD Program\Final Research\Sample Data" # Remove this line later
cntr_val = 1
for file_name in os.scandir(root_path):
    file_path = file_name.path
    print(file_path)
    dataset_df = pd.read_csv(file_path)
    db_row = {'File Name':[file_path]}
    
    num_classes = len(set(list(dataset_df.sound_file_class)))
    db_row = db_row | {'# classes':[num_classes]}

    data_size = int(dataset_df.shape[0])
    db_row = db_row | {'N':[data_size]}

    data_size = int(dataset_df.shape[1])
    db_row = db_row | {'# features':[data_size]}
    
    hilbert_flag = "Yes"
    if "_nht" in file_path:
        hilbert_flag = "No"

    db_row = db_row | {'Hilbert Transformed?':[hilbert_flag]}

    mfcc_flag = "Yes"
    if "_nMFCC" in file_path:
        mfcc_flag = "No"

    db_row = db_row | {'MFCC included?':[mfcc_flag]}

    sample_dur = "30 seconds"
    if "_5_sec" in file_path:
        sample_dur = "30 seconds"
    elif "_1_sec" in file_path:
        sample_dur = "1 second"

    db_row = db_row | {'Sample Duration':[sample_dur]}

    scaled_flag = "Yes"
    if "_ns" in file_path:
        scaled_flag = "No"
    
    db_row = db_row | {'Data Scaled?':[scaled_flag]}
    
    anomaly_flag = "Yes"
    if "_nat" in file_path:
        anomaly_flag = "No"
    
    db_row = db_row | {'Anomaly Treated?':[anomaly_flag]}
    
    # Determine the optimal parameters
    
    #1. Convert the sound_file_class label into a ordinal variable 
    X_raw = dataset_df
    X_raw['sound_file_class_num'] = X_raw.apply(lambda x: 1 if x.sound_file_class=='Q1' 
                                                        else (2 if x.sound_file_class=='Q2' 
                                                              else (3 if x.sound_file_class=='Q3' else 4)), axis=1)

    X = X_raw[X_raw.columns[~X_raw.columns.isin(['sound_file_name', 'sound_file_class','sound_file_class_num'])]]
    #print(list(X.columns))
    y,unique_vals = pd.factorize(X_raw['sound_file_class_num'], sort=True)
    
    best_score, best_parms = FindOptimalParm(X,y)
    
    db_row = db_row | {'learning_rate':[best_parms['learning_rate']]}
    db_row = db_row | {'n_estimators':[best_parms['n_estimators']]}
    db_row = db_row | {'Best Accuracy':[best_score]}
    
    
    data_df = pd.DataFrame(db_row)

    # Build feature dataset
    if cntr_val == 0:
        final_df = data_df
        #break
    else:
        final_df = pd.concat([final_df,data_df],axis=0)
    cntr_val +=1
final_df

D:\PhD Program\Final Research\Sample Data\DS_1_All_Feature_5_sec_nMFCC_nht_ns_nat.csv
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  20.6s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  22.0s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  20.4s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  20.5s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  20.4s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  40.9s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  40.7s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  40.9s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  40.8s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  42.8s
[CV] END ...............learning_rate=0.01

[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 2.7min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 2.7min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 2.7min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 2.7min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 2.7min
[CV] END ...............learning_rate=0.2, n_estimators=1000; total time= 3.4min
[CV] END ...............learning_rate=0.2, n_estimators=1000; total time= 3.4min
[CV] END ...............lear

[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1500; total time= 6.6min
[CV] END ..............learning_rate=0.05, n_estimators=1500; total time= 6.6min
[CV] END ..............learn

[CV] END ...............learning_rate=0.3, n_estimators=1500; total time= 6.6min
[CV] END ...............learning_rate=0.3, n_estimators=1500; total time= 6.6min
[CV] END ...............learning_rate=0.3, n_estimators=1500; total time= 6.6min
GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=1, n_splits=5, random_state=1),
             estimator=AdaBoostClassifier(), n_jobs=1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
                         'n_estimators': [100, 200, 500, 800, 1000, 1500]},
             scoring='accuracy', verbose=2)
Best scores:
Best: 0.633052 using {'learning_rate': 0.1, 'n_estimators': 1500}
D:\PhD Program\Final Research\Sample Data\DS_1_All_Feature_5_sec_nMFCC_nht_ys_nat.csv
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  20.4s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  20.8s
[CV] END ...............learning_rate=0.

[CV] END ................learning_rate=0.2, n_estimators=100; total time=  20.6s
[CV] END ................learning_rate=0.2, n_estimators=100; total time=  20.5s
[CV] END ................learning_rate=0.2, n_estimators=100; total time=  20.2s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  41.1s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  40.8s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  40.9s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  40.9s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  40.8s
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 1.7min
[CV] END ................lea

...

[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.2min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.1min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.1min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.2min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 3.4min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 3.4min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 3.4min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 3.4min
[CV] END ................learning_rate=0.2, n_estimators=800; total time= 3.4min
[CV] END ...............learning_rate=0.2, n_estimators=1000; total time= 4.3min
[CV] END ...............learning_rate=0.2, n_estimators=1000; total time= 4.3min
[CV] END ...............learning_rate=0.2, n_estimators=1000; total time= 4.3min
[CV] END ...............lear

[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ...............learning_rate=0.05, n_estimators=800; total time= 3.5min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1000; total time= 4.4min
[CV] END ..............learning_rate=0.05, n_estimators=1500; total time= 6.6min
[CV] END ..............learning_rate=0.05, n_estimators=1500; total time= 6.5min
[CV] END ..............learning_rate=0.05, n_estimators=1500; total time= 6.5min
[CV] END ..............learn

[CV] END ...............learning_rate=0.3, n_estimators=1500; total time= 6.5min
[CV] END ...............learning_rate=0.3, n_estimators=1500; total time= 6.5min
GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=1, n_splits=5, random_state=1),
             estimator=AdaBoostClassifier(), n_jobs=1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
                         'n_estimators': [100, 200, 500, 800, 1000, 1500]},
             scoring='accuracy', verbose=2)
Best scores:
Best: 0.597269 using {'learning_rate': 0.3, 'n_estimators': 800}
D:\PhD Program\Final Research\Sample Data\DS_1_All_Feature_5_sec_yMFCC_yht_ys_yat.csv
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  25.9s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  25.9s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  25.8s
[CV] END ...............learning_rate=0.0

[CV] END ................learning_rate=0.2, n_estimators=100; total time=  25.6s
[CV] END ................learning_rate=0.2, n_estimators=100; total time=  25.9s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  51.4s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  51.4s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  51.6s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  51.4s
[CV] END ................learning_rate=0.2, n_estimators=200; total time=  51.5s
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.1min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.1min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.2min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.1min
[CV] END ................learning_rate=0.2, n_estimators=500; total time= 2.1min
[CV] END ................lea

,File Name,# classes,N,# features,Hilbert Transformed?,MFCC included?,Sample Duration,Data Scaled?,Anomaly Treated?,learning_rate,n_estimators,Best Accuracy
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,390,No,No,30 seconds,No,No,0.01,500,0.536667
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,510,No,No,30 seconds,No,Yes,0.30,800,0.547778
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,390,No,No,30 seconds,Yes,No,0.01,500,0.536667
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,510,No,No,30 seconds,Yes,Yes,0.30,800,0.547778
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,385,Yes,No,30 seconds,No,No,0.01,1500,0.530000
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,505,Yes,No,30 seconds,No,Yes,0.20,800,0.555556
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,385,Yes,No,30 seconds,Yes,No,0.01,1500,0.530000
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,505,Yes,No,30 seconds,Yes,Yes,0.20,800,0.555556
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,510,No,Yes,30 seconds,No,No,0.05,200,0.541111
0,D:\PhD Program\Final Research\Sample Data\DS_1...,4,900,510,No,Yes,30 seconds,No,Yes,0.10,200,0.543333


In [16]:
final_df.shape

(16, 12)

In [18]:
final_df.to_csv("D:/PhD Program/Final Research/best_parm_data.csv",index=False)

In [ ]:
DS_1_All_Feature_5_sec_nMFCC_nht_ns_nat - 0.599809 using {'learning_rate': 0.3, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_nMFCC_nht_ns_yat - 0.633052 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_nMFCC_nht_ys_nat - 0.599809 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_nMFCC_nht_ys_yat - 0.633052 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_nMFCC_yht_ns_nat - 0.576907 using {'learning_rate': 0.2, 'n_estimators': 1000}
DS_1_All_Feature_5_sec_nMFCC_yht_ns_yat - 0.616189 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_nMFCC_yht_ys_nat - 0.577861 using {'learning_rate': 0.2, 'n_estimators': 1000}
DS_1_All_Feature_5_sec_nMFCC_yht_ys_yat - 0.616189 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_yMFCC_nht_ns_nat - 0.609511 using {'learning_rate': 0.2, 'n_estimators': 1000}
DS_1_All_Feature_5_sec_yMFCC_nht_ns_yat - 0.633052 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_yMFCC_nht_ys_nat - 0.611899 using {'learning_rate': 0.3, 'n_estimators': 500}
DS_1_All_Feature_5_sec_yMFCC_nht_ys_yat - 0.633052 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_yMFCC_yht_ns_nat - 0.595838 using {'learning_rate': 0.3, 'n_estimators': 800}
DS_1_All_Feature_5_sec_yMFCC_yht_ns_yat - 0.616189 using {'learning_rate': 0.1, 'n_estimators': 1500}
DS_1_All_Feature_5_sec_yMFCC_yht_ys_nat - 0.597269 using {'learning_rate': 0.3, 'n_estimators': 800}
DS_1_All_Feature_5_sec_yMFCC_yht_ys_yat - 0.616189 using {'learning_rate': 0.1, 'n_estimators': 1500}